
# Möbius Plateau Explorer v6 🔍

*Generated on 2025-07-14 12:04.*

Load your fine‑scan table (`bits, k, φ, λ, rel_err`), auto‑bucket rows into plateaux, and poke at them.

### What's new in v6  
* Tune plateau size with a live slider (10^‑x).  
* Quick ops now include φ, φ², 1/φ, Feigenbaum δ≈4.6692, α≈2.5029 (halves, squares, inverses).  
* Stack ops (`/9 ×φ ÷δ` …).  
* Mini charts update live so you can eyeball structure.  
* One‑click CSV / Excel export of whatever view you end on.

```
φ ≈ 0.00394344 ± 5×10⁻⁷   α⁻¹ ≈ 137.035999084  δ ≈ 4.669201609  α_feig ≈ 2.502907875
```

**How to use**
1. Paste *your* rows into the first code cell (`raw_rows = [...]`).
2. Run the notebook top→bottom.
3. In the widget  
   • toggle plateau(s) on/off  
   • tweak ε slider to merge / split plateaux  
   • chain ops in the box (`÷9 ×φ ÷δ` …)  
4. Use the last cell to download the filtered view as Excel.


In [5]:

import pandas as pd, numpy as np, ipywidgets as W

# ⬇️⬇️ PASTE / UPDATE YOUR RAW ROWS BELOW ⬇️⬇️
raw_rows = [
    (64,3.6,0.003942,0.0,0.4117525875906678269),
    (64,3.8,0.003942,0.0,0.4117525875906678269),
    (64,4.0,0.003942,0.0,0.4117525875906678269),
    (64,4.2,0.003942,0.0,0.4117525875906678269),
    (128,4.2,0.003942,0.0,0.4117525875906678271),
    (128,4.0,0.003942,0.0,0.4117525875906678271),
    (128,3.8,0.003942,0.0,0.4117525875906678271),
    (128,3.6,0.003942,0.0,0.4117525875906678271),
]

df = pd.DataFrame(raw_rows, columns=['bits','k','phi','lam','rel_err'])
display(df.head())


,bits,k,phi,lam,rel_err
0,64,3.6,0.003942,0.0,0.411753
1,64,3.8,0.003942,0.0,0.411753
2,64,4.0,0.003942,0.0,0.411753
3,64,4.2,0.003942,0.0,0.411753
4,128,4.2,0.003942,0.0,0.411753


In [6]:

# ==== Plateau detector with live ε slider ====
def tag(df_in, eps):
    out = df_in.copy()
    base = out['rel_err'].min()
    out['plateau'] = ((out['rel_err'] - base)/eps).astype(int)
    return out

eps_slider = W.FloatLogSlider(value=1e-5, base=10, min=-10, max=-1, step=0.25,
                              description='ε (rel_err)', continuous_update=False)

out_tbl = W.Output()

def refresh_tbl(change=None):
    with out_tbl:
        out_tbl.clear_output(wait=True)
        tagged = tag(df, eps_slider.value)
        display(tagged.groupby('plateau')['rel_err'].agg(['min','max','count']))
        display(tagged)

eps_slider.observe(refresh_tbl, 'value')
display(eps_slider, out_tbl)
refresh_tbl()


FloatLogSlider(value=1e-05, continuous_update=False, description='ε (rel_err)', max=-1.0, min=-10.0, step=0.25…

Output()

In [7]:
# ==== Quick‑op helper ====
alpha_inv = 137.035999084
phi_c = 0.00394344
phi2 = phi_c**2
alpha_c = alpha_inv**-1
delta = 4.669201609
delta2 = delta**2
alpha_feig = 2.502907875
alpha_feig2 = alpha_feig**2

def eval_tok(tok):
    consts = {
        '÷9' : 1/9, '×9' : 9,
        'φ' : phi_c, 'φ2':phi2,
        'α':alpha_inv, 'α2':alpha_inv**2, '/α':alpha_c,
        'δ':delta, 'δ2':delta2, '/δ':1/delta,
        'αf':alpha_feig, 'αf2':alpha_feig2, '/αf':1/alpha_feig,
    }
    return consts.get(tok.strip(), 1.0)

def apply_ops(series, op_str):
    tokens = op_str.split()
    out = series.copy()
    for tok in tokens:
        if tok.startswith('×'):
            val = eval_tok(tok[1:])
            out = out * val
        elif tok.startswith('÷') or tok.startswith('/'):
            val = eval_tok(tok[1:] if tok[0]=='÷' else tok[1:])
            out = out / val
        else:
            # bare token treated as multiply
            val = eval_tok(tok)
            out = out * val
    return out

# Apply the tag function to create the 'plateau' column in df
df = tag(df, eps_slider.value)

pl_sel = W.SelectMultiple(options=sorted(df['plateau'].unique()),
                          value=tuple(df['plateau'].unique()),
                          description='Plateaus')
op_box = W.Text(value='', placeholder='e.g. ÷9 ×φ ×φ2', description='Ops')
apply_btn = W.Button(description='Apply', button_style='success')
out_ops = W.Output()

def run_ops(_):
    with out_ops:
        out_ops.clear_output()
        chosen = list(pl_sel.value)
        view = df[df['plateau'].isin(chosen)].copy()
        if op_box.value.strip():
            view['rel_err'] = apply_ops(view['rel_err'], op_box.value)
        display(view.reset_index(drop=True))

apply_btn.on_click(run_ops)
display(W.HBox([pl_sel, op_box, apply_btn]), out_ops)

Output()

from matplotlib import pyplot as plt
_df_0['bits'].plot(kind='hist', bins=20, title='bits')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['k'].plot(kind='hist', bins=20, title='k')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2.plot(kind='scatter', x='bits', y='k', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['bits']
  ys = series['k']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_3.sort_values('bits', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('bits')
_ = plt.ylabel('k')

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['bits']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'bits'}, axis=1)
              .sort_values('bits', ascending=True))
  xs = counted['bits']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_13.sort_values('bits', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('bits')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['phi']
  ys = series['k']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_14.sort_values('phi', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('phi')
_ = plt.ylabel('k')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['phi']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'phi'}, axis=1)
              .sort_values('phi', ascending=True))
  xs = counted['phi']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_15.sort_values('phi', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('phi')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_16['bits'].plot(kind='line', figsize=(8, 4), title='bits')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_17['k'].plot(kind='line', figsize=(8, 4), title='k')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
from matplotlib import pyplot as plt
_df_8['k'].plot(kind='line', figsize=(8, 4), title='k')
plt.gca().spines[['top', 'right']].set_visible(False)

In [8]:

# ==== Export current view (raw, no ops) ====
file = 'plateaus_v6.xlsx'
df.to_excel(file, index=False)
print('Saved to', file)


Saved to plateaus_v6.xlsx
